# Cats-dog classification - CNN

This dataset was originally published by Microsoft Research and was later made available through Kaggle, at https://www.kaggle.com/c/dogs-vs-cats/data.

The dataset is hosted as a simple folder with filenames representing the label, so we might have to reorganize the dataset before we can use it.

PyTorch provides a neat abstraction for images with ImageFolder and DataLoader. PyTorch expects that data is stored in the following folder structure:

The dataset structure: <br>
**Cat images** - /input/train/cat/cat*.jpg <br>
**Dog Images** /input/train/dog/dog*.jpg <br>



To simplify the process, we have provided an organized structure, with images suitable for PyTorch experiments, at https://www.kaggle.com/jojomoolayil/catsvsdogs.

### Imports

In [2]:
# Import required libraries
import torch
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
import glob,os
import matplotlib.image as mpimg



## Check for GPU 

In [3]:
#Check if GPU is available
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print("Device:",device)

Device: cpu


In [4]:
new_path = "dogs-vs-cats/"

In [37]:

#train_df["img_name"] = os.listdir( os.path.join(new_path,"train","cat")) #os.listdir("train/")
file_names = os.listdir( os.path.join(new_path,"train","cat"))
file_names.extend(os.listdir( os.path.join(new_path,"train","dog")))
                  

In [38]:
import pandas as pd

train_df = pd.DataFrame(columns=["img_name","label"])

# create dataframe with column img_name with files_names, and label (0 or 1)
# if file name contains cat, label 0, otherwise 1 

train_df["img_name"] = ...

...
        
        
train_df.to_csv (r'train_csv.csv', index = False, header=True)

In [39]:
train_df

,img_name,label
0,cat.5077.jpg,0
1,cat.2718.jpg,0
2,cat.10151.jpg,0
3,cat.3406.jpg,0
4,cat.4369.jpg,0
...,...,...
19995,dog.9316.jpg,1
19996,dog.6025.jpg,1
19997,dog.8008.jpg,1
19998,dog.1992.jpg,1


In [40]:
from torch.utils.data import Dataset
import pandas as pd
import os
from PIL import Image
import torch

class CatsAndDogsDataset(Dataset):
    def __init__(self, root_dir, annotation_file, transform=None):
        self.root_dir = root_dir
        self.annotations = pd.read_csv(annotation_file)
        self.transform = transform

    def __len__(self):
        return ...

    def __getitem__(self, index):
        img_id = self.annotations.iloc[index, 0]
        img = ... 
        y_label = ...

        if self.transform is not None:
            img = self.transform(img)

        return (img, y_label)

## creating model

We are using transfer learning here. 

In [48]:
import torch.nn as nn
import torchvision.models as models

class CNN(nn.Module):
    def __init__(self, train_CNN=False, num_classes=1):
        super(CNN, self).__init__()
        self.train_CNN = train_CNN
        self.inception = models.inception_v3(pretrained=True)
        self.inception.fc = nn.Linear(self.inception.fc.in_features, num_classes)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)
        self.sigmoid = nn.Sigmoid()

    def forward(self, images):
        features = self.inception(images)
        return self.sigmoid(self.dropout(self.relu(features))).squeeze(1)

### Data Augmentation 

In [42]:
transform = transforms.Compose(
        [
            transforms.Resize((356, 356)),
            transforms.RandomCrop((299, 299)),
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
        ]
    )


### Hyperparammeters

In [43]:
num_epochs = 10
learning_rate = 0.00001
train_CNN = False
batch_size = 32
batch_size_validation = 128
shuffle = True
pin_memory = True
num_workers = 1

In [44]:

from torch.utils.data import DataLoader
import torchvision.transforms as transforms



### Create data loader and dataset 

In [46]:
dataset = CatsAndDogsDataset("dogs-vs-cats/train","train_csv.csv",transform=transform)
train_set, validation_set = torch.utils.data.random_split(dataset,[15000,5000])
train_loader = DataLoader(dataset=train_set, shuffle=shuffle, batch_size=batch_size,num_workers=num_workers,pin_memory=pin_memory)
validation_loader = DataLoader(dataset=validation_set, shuffle=shuffle, batch_size=batch_size_validation,num_workers=num_workers, pin_memory=pin_memory)

### Setting Optimizer

In [49]:
model = CNN().to(device)

criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for name, param in model.inception.named_parameters():
    if "fc.weight" in name or "fc.bias" in name:
        param.requires_grad = True
    else:
        param.requires_grad = train_CNN

Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /Users/atif/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100.0%


## Training 

In [ ]:
model.train()
for epoch in range(20):
        for imgs, labels in train_loader:
            imgs = imgs.to(device)
            labels = labels.to(device)
            outputs = model(imgs)
            ...
            
            
            